<center><h1>Panchal_Apoorva_HW3</h1></center>
<br>
<br>

Name: Apoorva Panchal
<br>
Github Username: ApoorvaPanchal
<br>
USC ID: 8685016290

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [12]:
from urllib.request import urlretrieve
import zipfile
import pandas as pd
from sklearn import metrics
from tabulate import tabulate
from prettytable import PrettyTable
import os
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats
import numpy as np

Get the AReM Data Set

In [13]:
# zip = zipfile.ZipFile('../data/AReM.zip', 'r').extractall('../data/')
folders = ['bending1', 'bending2', 'walking', 'standing', 'sitting', 'lying', 'cycling']

### (b) Test and Train Data

In [14]:
root = '../data/AReM/'
cs = ['time', 'avg_rss12','var_rss12','avg_rss13','var_rss13','avg_rss23','var_rss23']
test_df = pd.DataFrame()
train_df = pd.DataFrame()
# test_df = []
# train_df = []
# main_df = []
for i in ['bending1', 'bending2']:
    fs = os.listdir(root+i)
    for j in range(1, 3):
        df = pd.read_csv(root+i+'/dataset'+str(j)+'.csv', header=None,skiprows=5)
        test_df = pd.concat([test_df, df])

    for j in range(3, len(fs)+1):
        if(i=='bending2' and j==4):
            df = pd.read_csv(root+i+'/dataset'+str(j)+'.csv', header=None,skiprows=5, delim_whitespace=True)
        else:
            df = pd.read_csv(root+i+'/dataset'+str(j)+'.csv', header=None,skiprows=5)
        
        train_df = pd.concat([train_df, df])


for i in ['cycling', 'lying', 'sitting', 'standing', 'walking']:
    fs = os.listdir(root+i)
    for j in range(1, 4):
        df = pd.read_csv(root+i+'/dataset'+str(j)+'.csv',header=None,skiprows=5)
        test_df = pd.concat([test_df, df])


    for j in range(4, len(fs)+1):
        df = pd.read_csv(root+i+'/dataset'+str(j)+'.csv', header=None,skiprows=5)
        
        train_df = pd.concat([train_df, df])

train_df.columns = cs  
test_df.columns = cs

In [22]:
print(train_df.describe())
print(test_df.describe())

                time     avg_rss12     var_rss12     avg_rss13     var_rss13  \
count   33119.000000  33119.000000  33119.000000  33119.000000  33119.000000   
mean    59876.400254     39.005285      1.510208     14.166349      1.505252   
std     34641.049621      5.986658      2.082433      5.346573      1.648251   
min         0.000000      0.000000      0.000000      0.000000      0.000000   
25%     30000.000000     35.250000      0.430000     11.500000      0.430000   
50%     60000.000000     39.670000      0.500000     14.750000      0.830000   
75%     89875.000000     43.500000      1.920000     18.000000      2.290000   
max    119750.000000     56.250000     17.240000     32.750000     11.420000   

          avg_rss23     var_rss23  
count  33119.000000  33119.000000  
mean      15.669678      1.644033  
std        6.287338      1.646942  
min        0.000000      0.000000  
25%       12.000000      0.470000  
50%       15.670000      1.000000  
75%       18.750000      2.

Manually cleaned dataset9, dataset 14 in cycling for last comma at end of file, line 485

### (c) Feature Extraction

#### i. Research

The various types of time-domain features used in time series classification are:  
 * Mean
 * Median
 * Max, Min
 * Variance
 * Mode
 * Standar Deviation
 * Kurtosis
 * Skewness
 * Quartiles and IQR
 * Autocorrelation
 * Correlation
 * Segmentation and Differencing

#### ii. Extraction

In [16]:
instances = []
count = 0

for i in folders:
    fs = os.listdir(root+i)
    for f in range(1,len(fs)+1):
        if(i=='bending2' and f==4):
            df = pd.read_csv(root+i+'/dataset'+str(f)+'.csv', skiprows=5, delimiter=' ')
        else:
            df = pd.read_csv(root+i+'/dataset'+str(f)+'.csv', skiprows=5)
        curr = [count+1]
        for k in range(1,7):
            df_curr = df.iloc[:,k]
            curr.append(df_curr.min())
            curr.append(df_curr.max())
            curr.append(df_curr.mean())
            curr.append(df_curr.median())
            curr.append(df_curr.std())
            curr.append(df_curr.quantile(0.25))
            curr.append(df_curr.quantile(0.75))
        count+=1
        instances.append(curr)

In [17]:
features = ['Min', 'Max', 'Mean', 'Median', 'Standard Deviation', 'First Quartile', 'Third Quartile']
cols = ['Instance']
for i in range(1, 7):
    for j in features:
        cols.append(j+'_'+str(i))
instances_stats = pd.DataFrame(data=instances, columns=cols)

instances_stats

,Instance,Min_1,Max_1,Mean_1,Median_1,Standard Deviation_1,First Quartile_1,Third Quartile_1,Min_2,Max_2,...,Standard Deviation_5,First Quartile_5,Third Quartile_5,Min_6,Max_6,Mean_6,Median_6,Standard Deviation_6,First Quartile_6,Third Quartile_6
0,1,37.25,45.00,40.627662,40.50,1.477170,39.250,42.00,0.0,1.30,...,2.190586,33.000,36.000,0.0,1.92,0.569061,0.43,0.582568,0.000,1.300
1,2,38.00,45.67,42.811023,42.50,1.436515,42.000,43.67,0.0,1.22,...,1.992534,32.000,34.500,0.0,3.11,0.572276,0.43,0.601070,0.000,1.300
2,3,35.00,47.40,43.958580,44.33,1.557897,43.000,45.00,0.0,1.70,...,1.985273,35.450,36.500,0.0,1.79,0.494322,0.43,0.513547,0.000,0.940
3,4,33.00,47.75,42.169061,43.50,3.666929,39.100,45.00,0.0,3.00,...,3.853317,30.415,36.330,0.0,2.18,0.612088,0.50,0.523923,0.000,1.000
4,5,33.00,45.75,41.672526,41.75,2.242551,41.330,42.75,0.0,2.83,...,2.413538,28.415,31.250,0.0,1.79,0.384092,0.43,0.389176,0.000,0.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,84,27.00,44.33,36.530271,36.00,4.015354,33.250,39.75,0.0,10.43,...,3.090404,15.000,19.500,0.0,9.18,3.226994,2.87,1.771288,1.880,4.265
84,85,18.50,44.25,35.757056,36.00,4.618475,33.000,39.33,0.0,12.60,...,3.122793,14.000,18.125,0.0,8.81,3.056472,2.77,1.726063,1.795,4.045
85,86,19.00,43.75,35.882756,36.00,4.619270,33.000,39.50,0.0,11.20,...,3.541082,14.750,19.710,0.0,8.50,3.092088,2.92,1.627605,1.890,4.060
86,87,23.33,43.50,36.245115,36.75,3.825945,33.415,39.25,0.0,9.71,...,3.620323,15.750,21.000,0.0,11.15,3.536827,3.11,1.960835,2.170,4.625


#### iii. Standard Deviation

In [18]:
std_df = pd.DataFrame(columns=['Standard Deviation'])

for i in range(1, 43):
    std_df.loc[cols[i]] = instances_stats.iloc[:, i].std()

std_df

,Standard Deviation
Min_1,9.569975
Max_1,4.394362
Mean_1,5.336219
Median_1,5.439188
Standard Deviation_1,1.771195
First Quartile_1,6.155351
Third Quartile_1,5.141291
Min_2,0.000000
Max_2,5.062729
Mean_2,1.573780


In [19]:
std_df.sort_values('Standard Deviation')

,Standard Deviation
Min_4,0.000000
Min_2,0.000000
Min_6,0.045838
Standard Deviation_4,0.459151
Standard Deviation_6,0.517443
First Quartile_6,0.760409
First Quartile_4,0.842665
Standard Deviation_2,0.883932
First Quartile_2,0.945197
Standard Deviation_3,0.945207


In [20]:
bs_df = pd.DataFrame(columns=['Low', 'High', 'Diff'])

for i in range(1, 43):
    temp = bs.bootstrap(np.array(instances_stats.iloc[:, i]), stat_func=bs_stats.std, num_iterations=1000, alpha=0.1)
    # print(temp.lower_bound)
    bs_df.loc[cols[i]] = [temp.lower_bound, temp.upper_bound, temp.upper_bound-temp.lower_bound]
bs_df

,Low,High,Diff
Min_1,8.251684,10.805302,2.553617
Max_1,3.405473,5.415163,2.009690
Mean_1,4.776891,5.931793,1.154902
Median_1,4.883077,6.016196,1.133119
Standard Deviation_1,1.578711,1.955477,0.376766
First Quartile_1,5.635709,6.712913,1.077204
Third Quartile_1,4.455003,5.920611,1.465608
Min_2,0.000000,0.000000,0.000000
Max_2,4.687321,5.465999,0.778678
Mean_2,1.446946,1.732530,0.285584


In [21]:
bs_df.sort_values('Diff')

,Low,High,Diff
Min_4,0.000000,0.000000,0.000000
Min_2,0.000000,0.000000,0.000000
Standard Deviation_6,0.487423,0.550726,0.063303
Standard Deviation_4,0.429840,0.493367,0.063527
Min_6,0.013125,0.091154,0.078029
First Quartile_4,0.788581,0.906609,0.118028
First Quartile_6,0.704239,0.826592,0.122352
Standard Deviation_2,0.822468,0.963419,0.140952
Median_4,1.087286,1.230202,0.142917
Mean_4,1.103526,1.250750,0.147224


#### iv. Select Features

To select the most important features, we need to make sure that we include features that provide us variabilty and biasness as well.
The features with high standard deivation are more likely to be important as those values will have the variabilty, and the ones with a smaller confidence interval will provide us with bias, thus taking care of the bias-variance tradeoff

From the above values we can note :  
 * First Quartile has smaller confidence interval and a higher standard deviation
 * The same goes for the Third quartile
 * Median is an important feature as it can help us in dealing with outliers.  
  
Thus the selected features are First Quartile, Third Quartile and median


## 2. ISLR 3.7.4

### (a) Linear Train

The training RSS for cubic regression would be lower than that of linear regression. The reason being that since cubic regression has extra polynomial terms which in turn increase the degrees of freedom, it allows for a better fit of training data, thus resulting in reduced RSS. Also the number of predictors are directly proportional to the fit of the model resulting in the above conclusion


### (b) Linear Test

Since the true relationship between the data is linear, the simple linear regression is more likely to have lower test RSS. The reason being that we obtained lower train RSS for the cubic regression which means that it is more likely to overfit training data and give higher test RSS for unseen data. It is a trade-off that we make to obtain better results on test dataset, and hence, accept a slightly higher test RSS

### (c) Not Linear Train

The answer is similar to (a) as despite the true relationship of the data, more degrees of freedom and more predictors means that we obtain lower train RSS as the extra polynomial terms better fit the data. Thus, training RSS for cubic regression will be lower than that of simple linear regression

### (d) Not Linear Testing

Since we are not aware of the true relationship between the data, if it is more cubic than linear, the test RSS for cubic will be less than test RSS for simple linear regression as linear model will underfit the data and vice versa.